### Progress Update : Vanilla Anomaly Transformer Focusing on Association Discrepancy in Attention Part and Other Factors
We are focusing on the principle behide association discrepancy which includes Self-Attention (Series Asssociations) and Prior Association. In the first part, self-attention, we plotting out the self-attention to see how diffenrence between normal and abnormal points is, and how they are?

In [35]:
import os
import numpy as np

def load_files_to_arrays(directory):
    """
    Loads all files in the specified directory into a list of numpy arrays.
    Assumes each file contains data readable by np.load or np.loadtxt.
    """
    arrays = []
    for filename in sorted(os.listdir(directory)):
        filepath = os.path.join(directory, filename)
        if os.path.isfile(filepath):
            try:
                # Try loading as a numpy binary file first
                arr = np.load(filepath)
            except Exception:
                # If that fails, try loading as a text file
                arr = np.loadtxt(filepath)
            arrays.append(arr)
    return arrays

def load_files_to_dict(directory):
    """
    Loads all files in the specified directory into a dict of numpy arrays.
    Keys are filenames.
    """
    arrays = {}
    for filename in sorted(os.listdir(directory)):
        filepath = os.path.join(directory, filename)
        if os.path.isfile(filepath):
            try:
                arr = np.load(filepath)
            except Exception:
                arr = np.loadtxt(filepath)
            arrays[filename] = arr
    return arrays

In [36]:
# allinputs = load_files_to_dict("./checkpoints/PSM/100/values/inputs")
# alllabels = load_files_to_dict("./checkpoints/PSM/100/values/labels")
# allattentions = load_files_to_dict("./checkpoints/PSM/100/values/attentions")
# allcrits = load_files_to_dict("./checkpoints/PSM/100/values/crits")
# main_path = "C:\\Users\\Acer\\Documents\\cmu\\research\\Anomaly-Transformer\\checkpoints\\PSM\\110\\values"
main_path = "../checkpoints/experiments/my_exp/SAW/120/values"
allinputs = load_files_to_dict(os.path.join(main_path, "inputs"))
alllabels = load_files_to_dict(os.path.join(main_path, "labels"))
allattentions = load_files_to_dict(os.path.join(main_path, "attentions"))
allcrits = load_files_to_dict(os.path.join(main_path, "crits"))

In [37]:
import matplotlib.pyplot as plt
# attention layer 0
layer = 0
idx_batch = 0
file_name_template = f"batch_{idx_batch:06d}.npy"
attention_key = f"layer{layer}_batch_{idx_batch:06d}.npy"
# get data
attention_matrix = allattentions[attention_key]
crits_data = allcrits[file_name_template]
input_data = allinputs[file_name_template]
if alllabels:
    label_data = alllabels[file_name_template]


### Series Association

In [38]:
def plot_attention_heatmap(attention_matrix, layer, batch_idx, labels=None):
    # create save plots path
    save_path = os.path.join(main_path, "plots", f"layer{layer}",f"batch_{batch_idx:06d}")
    os.makedirs(save_path, exist_ok=True)
    # plot attention matrix
    # attns shape: (batch_size, n_heads, win_size, win_size)
    sample_idx = 0  # which sample in the batch to visualize (0..batch_size-1)
    for sample_idx in range(attention_matrix.shape[0]):
        B, H, S1, S2 = attention_matrix.shape
        if labels is not None:
            label = labels[sample_idx]
            marked_indices = np.where(label == 1)[0]
        # Create a single-row figure with H columns (one subplot per head)
        fig, axes = plt.subplots(1, H, figsize=(8 * H, 8))
        # If there's only one head, axes may not be an array
        if H == 1:
            axes = [axes]

        for h in range(H):
            ax = axes[h]
            attn = attention_matrix[sample_idx, h]
            im = ax.imshow(attn, aspect='auto', cmap='viridis')
            for idx in marked_indices:
                ax.axhline(y=idx, color='red', linestyle='-', linewidth=1, alpha=0.7)
                ax.axvline(x=idx, color='red', linestyle='-', linewidth=1, alpha=0.7)
            ax.set_title(f'Head {h}')
            ax.axis('off')

        # Add a single colorbar for the row of heads
        # fig.colorbar(im, ax=axes, fraction=0.02, pad=0.02)
        plt.tight_layout()
        # plt.show()
        # plt.close() 
        filename = os.path.join(save_path, f"sample{sample_idx:06d}.png")    
        fig.savefig(filename)
        plt.close() 

def plot_crits(crits_data, input_data, batch_idx, labels=None):
    # create save plots path
    save_path = os.path.join(main_path, "plots", f"crits",f"batch_{batch_idx:06d}")
    os.makedirs(save_path, exist_ok=True)
    # plot crits
    sample_idx = 0  # which sample in the batch to visualize (0..batch_size-1)
    for sample_idx in range(crits_data.shape[0]):
        crits = crits_data[sample_idx]
        inputs = input_data[sample_idx]
        fig, ax = plt.subplots(2, 1, figsize=(12, 6))
        ax[0].plot(inputs, label='Input')
        ax[1].plot(crits, label='Crits')
        if labels is not None:
            label = labels[sample_idx]
            marked_indices = np.where(label == 1)[0]
            for idx in marked_indices:
                ax[0].axvline(x=idx, color='red', linestyle='-', linewidth=1, alpha=0.7)
                ax[1].axvline(x=idx, color='red', linestyle='-', linewidth=1, alpha=0.7)
        # fig.suptitle(f'Crits for Sample {sample_idx} in Batch {batch_idx}')
        ax[0].set_xlabel('Time Step')
        ax[0].set_ylabel('Input Value')
        ax[1].set_xlabel('Time Step')
        ax[1].set_ylabel('Crit Value')
        # ax[1].set_ylim(0, 4)
        # fig.legend()
        filename = os.path.join(save_path, f"sample{sample_idx:06d}_crits.png")    
        plt.savefig(filename)
        plt.close()

In [ ]:
plot_crits(crits_data,input_data, idx_batch, labels=label_data)
plot_attention_heatmap(attention_matrix, layer, idx_batch, labels=label_data)

Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x103442f90>>
Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.11/site-packages/ipykernel/ipkernel.py", line 775, in _clean_thread_parent_frames
    def _clean_thread_parent_frames(

KeyboardInterrupt: 


In [34]:
import pandas as pd
# data = pd.read_csv('../../../Downloads/test_label.csv')
data = pd.read_csv('../dataset/SAW/test_label.csv')
data['label'].value_counts()

label
0    14564
1      436
Name: count, dtype: int64

### Prior Attention

### Association Discrepancy